In [1]:
using Flux

In [2]:
using Random

In [7]:
collect(1:10)

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [ ]:
# Generate dataset of NN matricies W and H and their product V
#set seed
seed!(314)

# V ∈ R^(m × n), and r is the (low) rank of V
m, n = 15, 15
r = 5
true_W = abs.(randn((m, r)))

train_set_size = 5
test_set_size  = 1

train_H, train_V = collect(1:train_set_size), collect(1:train_set_size) #TODO optimize later
test_H,  test_V  = collect(1:test_set_size),  collect(1:test_set_size)  #optimize later

for i ∈ 1:train_set_size
    H = abs.(randn((r, n)))
    V = true_W*H
    # save H and V into a data set
    train_H[i] = H
    train_V[i] = V
end

for i ∈ 1:test_set_size
    H = abs.(randn((r, n)))
    V = true_W*H
    # save H and V into a data set
    test_H[i] = H
    test_V[i] = V
end

# LATER can add small noise

In [ ]:
# Define the model (copied from https://fluxml.ai/Flux.jl/stable/models/advanced/)
struct MyModel
    chain::Chain
end
  
function (m::MyModel)(x)
    # Arbitrary code can go here, but note that everything will be differentiated.
    # Zygote does not allow some operations, like mutating arrays.
    return m.chain(x)
end
  
# Call @functor to allow for training. Described below in more detail.
Flux.@functor MyModel

update_H(H) = H .* (W'*V) ./ (W'*W*H) # can add regularization later
# make W the trainable parameters
# start with some random (but fixed for all inputs?) guess for here
# note the model will take the form:
h_guess = model(V_input)

In [8]:
[1,2,3][1]

1

In [ ]:
# or could define it like this?
struct MyLayer
    W
end

function MyLayer((m,r)::Pair{<:Integer, <:Integer})
    W = abs.randn(m,r) # TODO adjust initialization
    MyLayer(W)
end

function (a::MyLayer)(H, V)
    W = a.W
    return (H .* (W'*V) ./ (W'*W*H), V) #one update step
    # TODO can add regularization later
end

@functor MyLayer

struct MyModel
    chain::Chain
end
  
function (m::MyModel)(V, H_guess=ones(r,n))
    H_out, V_out = m.chain(H_guess, V)
    return H_out #just care about the first entry
end
  
# Call @functor to allow for training. Described below in more detail.
@functor MyModel

my_layer = MyLayer(m,r)
chain = Chain(my_layer, my_layer, my_layer) #three layer network, each layer shares parameters
model(V) = MyModel(chain)

In [ ]:
# Train the model

In [ ]:
# Evaluate the model performance on the training and testing set

In [ ]:
# Observe how close the learned weights ̂W and the true W are